# Rainier Financial Model API User Guide

Below is the user guide for the Rainier Financial Model, written by Alex Warfel, CFA. This notebook contains Python code showing how you can interact with the API. You can also use Postman and other tools. Use is subject to certain restrictions. Always consult a financial professional before implementing or acting on the results of this model.

- [API and project details](https://rainierfm.com/)
- [Detailed API documentation](https://rainierfm.com/docs)

In [1]:
import requests, json
import pandas as pd
import matplotlib.pyplot as plt

# Initial setup and Credentialling

First, we need to define the base URL that is used for connecting to the API. Since this project is still early on, the base url could change. 

In [2]:
base_url = "https://rainierfm.com"

Next, you can hardcode your credentials here which will assist later on in working with the API. Remember, don't share these credentials with anyone other than you. 

In [3]:
email = "example@youremail.com"
password = "a_super_complex_password_22"

# Confirming the API is healthy

First, we will do a health check of the API and confirm that it is operating properly before we connect to it. The response should look like:

```
200
{'status': 'healthy'}
```

In [ ]:
health_check_url = f"{base_url}/health/"

response = requests.get(health_check_url)
print(response.status_code)
response.json()

# Account Registration

Next, we'll create your account. This process creates an account for you in our third party identity provider. Reminder: we don't store your email address alongside any financial information. If things are successful, the response you can expect is:

```
200
{'message': 'User registered successfully'}
```

In [ ]:
register_url = f"{base_url}/register/"
payload = {
  "email": email,
  "password": password
}
response = requests.post(register_url, json=payload)
print(response.status_code)
response.json()

# Login and collect an access token

Once you've created an account successfully, you will want to login and get an access token. The token itself is how the API will determine who you are and what you have access to. Do not share this token with anyone, and you will need to create a new token after some number of hours. This is to improve security. A successful response will look like:

```
200
{'access_token': <some_token_string>}
```

In [ ]:
login_url = f"{base_url}/login/"
payload = {
    "email": email,
    "password": password
}

headers = {
    "Content-Type": "application/json"
}

response = requests.post(login_url, json=payload, headers=headers)
print(response.status_code)
response.json()

After that's done, just assign your access token to some variable so it can easily be used throughout the rest of your session. You will also now need to use headers to interact with the API. The headers are sent alongside your API requests and will pass your token onto the API which is used to determining who you are. 

In [7]:
access_token = response.json()["access_token"]
access_token

# add the access token to the headers
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {access_token}"
}

# Add inputs to the model and initiate a model run

Next, you'll want to send along some financial details to the model so a model run can begin. Below is what the typical request body will look like. Once you submit these details, the model will begin to run. It takes about two minutes for a run to complete. If your results are accepted, this is the response you can expect:

```
201
{'message': 'Model inputs added successfully and Lambda function invoked'}
```

## NOTE
If you have not been approved to use the model, you will see an error indicating you are not a valid user. That will look like:

```
403
{'error': 'User does not have the required valid_user role'}
```

In order to resolve this, you will need to reach out to alexwarfel@gmail.com to resolve this. 

In [ ]:
add_model_inputs_url = f"{base_url}/add_model_inputs/"

payload = {
    "gross_current_income": 65000,
    "gross_current_savings_rate": 0.25,
    "assets": 1000,
    "savings": 5000,
    "tax_rate": 0.30,
    "birthday": "2000-11-29",
    "current_debt_accounts": [
        {
            "balance": 2500,
            "interest_rate": 0.21,
            "minimum_payment": 100,
            "name": "Credit Card"
        },
        {
            "balance": 90000,
            "interest_rate": 0.05,
            "minimum_payment": 200,
            "name": "Student Loan"
        },
        {
            "balance": 10000,
            "interest_rate": 0.07,
            "minimum_payment": 100,
            "name": "Car Loan"
        }
    ],
    "gender": "M",
    "percentage_of_nominal_spending_for_housing": 0.25,
    "initial_retirement_age": 65,
    "non_retirement_goals": [
        {
            "age": 27,
            "amount_in_real_terms": 4000,
            "chance_of_success": 0.90,
            "name": "Trip to Europe"
        },
        {
            "age": 35,
            "amount_in_real_terms": 25000,
            "chance_of_success": 0.75,
            "name": "Buy a boat"
        }
    ]
}

response = requests.post(add_model_inputs_url, headers=headers, data=json.dumps(payload))
print(response.status_code)
response.json()

# Check the status of your model runs

After you've submitted some inputs, you can check the status of those model runs. A typical response will look like:

```
200
[{'model_id': <some_model_id_string>,
  'model_input_dt': '2024-09-14T15:36:18',
  'model_start_runtime_dt': '2024-09-14T15:36:21',
  'model_output_dt': '2024-09-14T15:38:07',
  'status': 'complete',
  'time_since_submission': '6m 16s',
  'model_runtime': '1m 46s'}]
```

In [ ]:
check_model_metadata = f"{base_url}/view_model_metadata/"
payload = {
  "email": email,
  "password": password
}

response = requests.post(check_model_metadata, headers=headers, data=json.dumps(payload))
print(response.status_code)
response.json()

# Checking the results of the model

The ```model_id``` from the above request will be important. That is how you will access the results of a model. You can pass that ```model_id``` in to this next endpoint to see a JSON object containing the inputs, outputs, and other details related to the model. Refer to the documentation for the outputs. 

In [ ]:
model_id = response.json()[0]["model_id"]

check_model_results = f"{base_url}/view_model_results/"
payload = {
  "email": email,
  "model_id": model_id,
  "password": password
}

response = requests.post(check_model_results, headers=headers, data=json.dumps(payload))
print(response.status_code)
model_output = response.json()[0]
model_output

# Working with the model outputs

Now that you have obtained the results of the model, here is how you can review and analyze them. The first layer of the JSOn object contains the outputs of two broad models. The first model calculates the optimal savings rate, and teh second model calculates the distributional outcomes assuming that optimal savings rate. 

In [ ]:
model_output_results = model_output['model_output']
model_output_results.keys()

# Optimal savings rates
First, we'll look at the optimized savings rate. 

In [ ]:
monte_carlo_optimize_savings_rate = model_output_results['monte_carlo_optimize_savings_rate']
monte_carlo_optimize_savings_rate

In [ ]:
monte_carlo_optimize_savings_rate_df = pd.DataFrame(monte_carlo_optimize_savings_rate)
monte_carlo_optimize_savings_rate_df

The below plot will show how your chance of meeting your retirement and non-retirement goals based on the different savings rates we attempted. 

In [ ]:
monte_carlo_optimize_savings_rate_df.plot(x='Savings Rate', y='Success Rate', kind='area')

# Distributional analysis
Below are the types of analyses that are already completed from the model. Most of the time, things are broken down by a column called ```name``` which represents the user's age. Things are also typically broken out by quantile, so you will often see ```<column_name_qXX>``` where XX represents the quantile number. The monte carlo is run hundreds of times, so it makes sense to provide the distribution results. 

In [ ]:
model_output_results['monte_carlo_distributions'].keys()

In [37]:
monte_carlo_distributions = model_output_results['monte_carlo_distributions']

networth_chart = monte_carlo_distributions['networth_chart']
recommendations = monte_carlo_distributions['recommendations']
full_distributions = monte_carlo_distributions['full_distributions']
cashflow_composition = monte_carlo_distributions['cashflow_composition']
lifetim_salary_and_spending = monte_carlo_distributions['lifetime_salary_and_spending']

In [ ]:
# Convert to DataFrame
full_distributions_df = pd.DataFrame(full_distributions)
full_distributions_df.rename(columns={'name':'age'}, inplace=True)
full_distributions_df = full_distributions_df.set_index('age')
full_distributions_df

In [ ]:
# Remove the quantile denotations to see what kind of columns are included the in the DataFrame
full_distributions_df_columns = full_distributions_df.columns.str.replace(r'_q\d{2}$', '', regex=True)
full_distributions_df_columns = full_distributions_df_columns.unique()
full_distributions_df_columns

In [ ]:
# Filter the dataframe to remove 0 values
filtered_df = full_distributions_df[['Real Salary_q10', 'Real Salary_q25', 'Real Salary_q50', 'Real Salary_q75', 'Real Salary_q90']].replace(0, pd.NA).dropna()

# Remove the first record because it is a partial year
filtered_df = filtered_df.drop(filtered_df.index[0])

# Plot the filtered dataframe
filtered_df.plot()

In [ ]:
cashflow_composition_df = pd.DataFrame(cashflow_composition)
cashflow_composition_df

In [ ]:
cashflow_composition_df_columns = cashflow_composition_df.columns.str.replace(r'_q\d{2}$', '', regex=True)
cashflow_composition_df_columns = cashflow_composition_df_columns.unique()
cashflow_composition_df_columns

In [ ]:
# Filter the dataframe to remove 0 values
filtered_df = cashflow_composition_df[['Real Amount Returned by Market_q10', 'Real Amount Returned by Market_q25', 'Real Amount Returned by Market_q50', 'Real Amount Returned by Market_q75', 'Real Amount Returned by Market_q90']].replace(0, pd.NA).dropna()

# Remove the first record because it is a partial year
filtered_df = filtered_df.drop(filtered_df.index[0])

# Plot the filtered dataframe
filtered_df.plot()

In [33]:
def save_to_csv(df, filename):
    df.to_csv(filename)
    print(f"Saved to {filename}")

# Change password

In the event you believe your password is compromised and you would like to change it, you can do so with the below endpoint. A successful response will look like:

```
200
{'message': 'Password changed successfully'}
```

In [ ]:
# change_password_url = f"{base_url}/change_password/"
# payload = {
#     "email": email,
#     "new_password": "new_password1234",
#     "old_password": password
# }

# response = requests.post(change_password_url, headers=headers, data=json.dumps(payload))
# print(response.status_code)
# response.json()

# Delete account

In the event you would like to remove your personally identifiable information from the API, you can delete your account. Note that you will have to re-register in order to use the API again. The only response you can expect if things are successful is:

```
204
```

In [ ]:
# delete_account_url = f"{base_url}/delete_account/"
# payload = {
#     "email": email,
#     "password": password
# }

# response = requests.delete(delete_account_url, headers=headers, data=json.dumps(payload))
# print(response.status_code)